In [1]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client['apartments_com']
table = db['austin_apartments']

# Import BeautifulSoup
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import time
import datetime
from pymongo import MongoClient
import pprint



In [39]:
# borrowed and adapted code from Chuanxiu Xiong
# https://github.com/chuanxiuXiong/apartments.com-scraper/blob/master/scraper.py

request_header = {
#         'Accept': "application/json, text/javascript, */*; q=0.01",
#         'Accept-Encoding': "gzip, deflate, br",
#         'Accept-Language': "en-US, en; q=0.8, zh-Hans-CN; q=0.5, zh-Hans; q=0.3",
#         'Cache-Control': "no-cache",
#         'Content-Type': "application/json",
#         'Host': "www.apartments.com",
#         'Origin': "https://www.apartments.com",
#         'Referer': "https://www.apartments.com/",
        'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36 Edge/17.17134",
#         'X-Requested-With': "XMLHttpRequest",
        }

In [40]:
# get the first page of apartments in Austin
base_url = "https://www.apartments.com/austin-tx/1/"

webpage = requests.get(base_url, headers=request_header)

webpage.text[:1000]

'\r\n\r\n<!DOCTYPE html>\r\n<html id="appContainer" lang="en" data-placeholder-focus="false">\r\n<head>\r\n    <meta charset="utf-8" />\r\n    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no" />\r\n    <meta name="format-detection" content="telephone=no" />\r\n    <meta name="format-detection" content="email=no" />\r\n    <meta name="HandheldFriendly" content="true" />\r\n    <title> Apartments for Rent in Austin TX | Apartments.com</title>\r\n    <link rel="icon" href="/a/8a55f9/favicon.ico">\r\n        <link rel="canonical" href="https://www.apartments.com/austin-tx/" />\r\n        <meta name="description" content="See all 24,365 apartments in Austin, TX currently available for rent. Each Apartments.com listing has verified availability, rental rates, photos, floor plans and more." />\r\n    <meta name="language" content="en" />\r\n    <meta http-equiv="language" content="en">\r\n    <meta http-equiv="content-language" content